In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:

test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
data_train_items = data_train['item_id'].value_counts().to_frame('count').reset_index()

In [5]:
data_train_items.rename(columns={'index': 'item_id', 'count': 'weight'}, inplace=True)
data_train_items['weight'] = data_train_items['weight'] / data_train.shape[0]
data_train_items.head()

,item_id,weight
0,1082185,0.011467
1,6534178,0.007700
2,1029743,0.005590
3,995242,0.004803
4,1106523,0.003828


In [6]:
data_train_items.shape

(86865, 2)

In [7]:
data_train_items['weight'].sum()

1.0

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [8]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    items_weights = items_weights
    recs = np.random.choice(items_weights['item_id'], size=n, replace=False, p=items_weights['weight'])
    
    return recs.tolist()

При чтении из csv происходят очень большие изменения содержимого - если поля с нашими рекомендациями списки просто преобразованы в строку (легко решаемо), то массив в поле 'actual' меняется очень сильно. городить тонны сток кода зазря не вижу смысла - просто фрейм с вебинара перезаписал в pickle - в него пишет без искажений (правда к этому варианту я пришел только спустя два часа попыток привести в порядок то, что читает из csv))))

In [46]:
result = pd.read_pickle('predictions_basic.pickle')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[861558, 840883, 891849, 1052721, 1020516]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6961519, 1037892, 969039, 5581971, 981427]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]"


In [47]:
%%time

# your_code
result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(data_train_items, n=5))

result.head(2)

CPU times: user 1.77 s, sys: 0 ns, total: 1.77 s
Wall time: 1.77 s


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[861558, 840883, 891849, 1052721, 1020516]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[1082185, 1029743, 995785, 1004906, 1081177]","[1911669, 951590, 967373, 9835509, 13512060]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6961519, 1037892, 969039, 5581971, 981427]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[1082185, 1098066, 6534178, 1127831, 1068719]","[962229, 922460, 6534077, 5568613, 963727]"


In [11]:
result.dtypes

user_id                            int64
actual                            object
random_recommendation             object
popular_recommendation            object
itemitem                          object
cosine                            object
tfidf                             object
own_purchases                     object
weighted_random_recommendation    object
dtype: object

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [51]:
from metrics import precision_at_k, recall_at_k

for name_col in result.columns[1:]:
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0005:random_recommendation
0.1552:popular_recommendation
0.1369:itemitem
0.1329:cosine
0.139:tfidf
0.2019:own_purchases
0.0184:weighted_random_recommendation


/home/mrbrain/Загрузки/webinar_2/metrics.py:20: RuntimeWarning: invalid value encountered in long_scalars
  return flags.sum() / len(recommended_list)


Рекомендации на основе собственных покупок по прежнему в лидерах, случайные взвешенные рекомендации лучше разве что просто случайных (почти в 37 раз между прочим)))